# 問1

前章の練習問題・問3では、`max_depth=2` と設定しましたが、評価用データの正解率は50%、学習用データの正解率は57.8%でした。そもそも学習用の正解率があまり高くないので、`max_depth` を増やしてみましょう。

評価用データの正解率は50%、学習用データの正解率は84.4%です。学習用の正解率はそこそこですが、評価用の正解率はあまり高くありません。


In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

X = pd.read_excel("data/chapter08_exercise.xlsx")
X.head()

,世帯年間収入,合計特殊出生率,地方,都道府県,人口15歳未満比率,15～64歳比率,65歳以上比率,人口男女比率（20～39歳）,インターネット利用者の割合
0,4488,1.19,0,北海道,0.107619,0.573714,0.318667,1.005929,88.4
1,4952,1.28,1,青森県,0.106742,0.560995,0.333066,1.066667,80.0
2,5282,1.39,1,岩手県,0.111654,0.557457,0.330888,1.076190,85.9
3,5702,1.27,1,宮城県,0.117953,0.599306,0.282741,1.056680,86.2
4,5274,1.31,1,秋田県,0.098344,0.530021,0.371636,1.068493,82.8


In [6]:
y = X["地方"]
z = X["都道府県"]

X2 = X.copy().drop(columns=["地方", "都道府県"], axis=1)

In [8]:
[
    X_train,
    X_test,
    y_train,
    y_test
] = sklearn.model_selection.train_test_split(X2, y, test_size=0.04, random_state=0)

In [10]:
model = DecisionTreeClassifier(max_depth=4, random_state=0)
model.fit(X_train, y_train)

score_te = model.score(X_test, y_test)
print("正解率", score_te * 100, "%")

score_tr = model.score(X_train, y_train)
print("正解率", score_tr * 100, "%")

正解率 50.0 %
正解率 84.44444444444444 %


# 問2

評価用データの正解率は50%、学習用データの正解率は95.6%です。

正解は、4:近畿、5:中国ですが、予想は、4:近畿、3:中部となり、半分しか正解していません。

確かに決定木に比べると、ランダムフォレストは学習用の正解率が大幅に改善しています。しかし、評価用の正解率はあまり高くないままです。いわゆる過学習の状態になっています。


In [13]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=4,
    random_state=0
)
model_rf.fit(X_train, y_train)

score_te = model_rf.score(X_test, y_test)
print("正解率", score_te * 100, "%")

score_tr = model_rf.score(X_train, y_train)
print("正解率", score_tr * 100, "%")

正解率 50.0 %
正解率 95.55555555555556 %


In [14]:
y_test_pred = model_rf.predict(X_test)

test_results = pd.DataFrame({
    "y_test": y_test,
    "y予想": y_test_pred
})

test_results_all = pd.concat(
    [X_test, test_results],
    axis=1
)
test_results_all.head()

,世帯年間収入,合計特殊出生率,人口15歳未満比率,15～64歳比率,65歳以上比率,人口男女比率（20～39歳）,インターネット利用者の割合,y_test,y予想
28,5452,1.22,0.118797,0.568421,0.313534,0.945736,89.3,4,4
33,5323,1.43,0.127675,0.578816,0.293509,1.083032,87.8,5,3


# 問3

今回は、学習用のデータが45と少ないことが低い正解率の一因と推測されます。45は学習に十分な数とは言えません。単年ではなく、複数年にすれば数が増えます。

また、データの特徴量を変えることで正解率を上げられる可能性もあるでしょう。たとえば、インターネット利用者の割合と15~64歳比率や65歳以上比率のように、相関が強い特徴量があるからです。相関が強いと同じような情報しか提供できません。

いずれにしても、学習用の正解率が高くても、評価用の正解率が今一つということがあるのを分かっていただけたでしょうか。予想は思っているより難しかったりするのです。


In [15]:
cor = X2.corr()
df = pd.DataFrame(cor)
df.head()

,世帯年間収入,合計特殊出生率,人口15歳未満比率,15～64歳比率,65歳以上比率,人口男女比率（20～39歳）,インターネット利用者の割合
世帯年間収入,1.000000,-0.304883,-0.155290,0.476142,-0.360335,0.600375,0.321421
合計特殊出生率,-0.304883,1.000000,0.688812,-0.431164,0.134164,-0.019069,-0.366069
人口15歳未満比率,-0.155290,0.688812,1.000000,0.209820,-0.529982,-0.241309,0.218256
15～64歳比率,0.476142,-0.431164,0.209820,1.000000,-0.940195,0.150902,0.756309
65歳以上比率,-0.360335,0.134164,-0.529982,-0.940195,1.000000,-0.045300,-0.730579


特徴量の間の相関を調べて、その結果をエクセルで出力します。 

In [16]:
df.to_excel("data/output.xlsx")

`df.to_excel`に保存先のパスを指定することでxlsx形式でデータフレームをエクスポートできます。ここで紹介しているコードを実行した場合，サブフォルダ`data`の中に`chapter08_output.xlsx`として保存されます（保存時にエラーが出る場合は，必要なライブラリがインストールされていないことが考えられます。この場合はopenpyxlなどのライブラリをインストールしてみましょう）。